# Termination 

In the previous section, we explored how to define agents, and organize them into teams that can solve tasks. However, a run can go on forever, and in many cases, we need to know _when_ to stop them. This is the role of the termination condition.

AgentChat supports several termination condition by providing a base {py:class}`~autogen_agentchat.base.TerminationCondition` class and several implementations that inherit from it.

A termination condition is a callable that takes a sequece of {py:class}`~autogen_agentchat.messages.AgentEvent` or {py:class}`~autogen_agentchat.messages.ChatMessage` objects **since the last time the condition was called**, and returns a {py:class}`~autogen_agentchat.messages.StopMessage` if the conversation should be terminated, or `None` otherwise.
Once a termination condition has been reached, it must be reset by calling {py:meth}`~autogen_agentchat.base.TerminationCondition.reset` before it can be used again.

Some important things to note about termination conditions: 
- They are stateful but reset automatically after each run ({py:meth}`~autogen_agentchat.base.TaskRunner.run` or {py:meth}`~autogen_agentchat.base.TaskRunner.run_stream`) is finished.
- They can be combined using the AND and OR operators.

```{note}
For group chat teams (i.e., {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`,
{py:class}`~autogen_agentchat.teams.SelectorGroupChat`, and {py:class}`~autogen_agentchat.teams.Swarm`),
the termination condition is called after each agent responds.
While a response may contain multiple inner messages, the team calls its termination condition just once for all the messages from a single response.
So the condition is called with the "delta sequence" of messages since the last time it was called.
```

Built-In Termination Conditions: 
1. {py:class}`~autogen_agentchat.conditions.MaxMessageTermination`: Stops after a specified number of messages have been produced, including both agent and task messages.
2. {py:class}`~autogen_agentchat.conditions.TextMentionTermination`: Stops when specific text or string is mentioned in a message (e.g., "TERMINATE").
3. {py:class}`~autogen_agentchat.conditions.TokenUsageTermination`: Stops when a certain number of prompt or completion tokens are used. This requires the agents to report token usage in their messages.
4. {py:class}`~autogen_agentchat.conditions.TimeoutTermination`: Stops after a specified duration in seconds.
5. {py:class}`~autogen_agentchat.conditions.HandoffTermination`: Stops when a handoff to a specific target is requested. Handoff messages can be used to build patterns such as {py:class}`~autogen_agentchat.teams.Swarm`. This is useful when you want to pause the run and allow application or user to provide input when an agent hands off to them.
6. {py:class}`~autogen_agentchat.conditions.SourceMatchTermination`: Stops after a specific agent responds.
7. {py:class}`~autogen_agentchat.conditions.ExternalTermination`: Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., "Stop" buttons in chat interfaces).
8. {py:class}`~autogen_agentchat.conditions.StopMessageTermination`: Stops when a {py:class}`~autogen_agentchat.messages.StopMessage` is produced by an agent.

To demonstrate the characteristics of termination conditions, we'll create a team consisting of two agents: a primary agent responsible for text generation and a critic agent that reviews and provides feedback on the generated text.

In [1]:
%%capture
# capture magic suppresses install output
!poetry add autogen_core autogen_ext autogen_agentchat openai tiktoken

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

Let's explore how termination conditions automatically reset after each `run` or `run_stream` call, allowing the team to resume its conversation from where it left off.

In [3]:
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Rain-kissed cobblestones,  
Whispers weave through autumn leaves—  
Eiffel shrouded gray.
---------- critic ----------
This is a beautiful and evocative Haiku that captures the essence of Parisian weather. The imagery of "Rain-kissed cobblestones" immediately sets a rainy, atmospheric scene. The second line, "Whispers weave through autumn leaves," suggests movement and a gentle, possibly melancholic feel that aligns well with typical autumn weather. The final line, "Eiffel shrouded gray," ties in the iconic Paris landmark with the gray skies, completing the picture vividly. 

Feedback: Consider varying the use of adjectives or adding a personal touch or emotion to deepen the imagery even further. This could enhance the connection for readers who may not have experienced the Paris weather firsthand.

Otherwise, excellently composed!  

Let me know if you'd like to refine or expand f

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=24), content='Rain-kissed cobblestones,  \nWhispers weave through autumn leaves—  \nEiffel shrouded gray.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=155), content='This is a beautiful and evocative Haiku that captures the essence of Parisian weather. The imagery of "Rain-kissed cobblestones" immediately sets a rainy, atmospheric scene. The second line, "Whispers weave through autumn leaves," suggests movement and a gentle, possibly melancholic feel that aligns well with typical autumn weather. The final line, "Eiffel shrouded gray," ties in the iconic Paris landmark with the gray skies, completing the picture vividly. \n\nFeedback: Consider varying the use of adjectives or adding 

The conversation stopped after reaching the maximum message limit. Since the primary agent didn't get to respond to the feedback, let's continue the conversation.

In [4]:
# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream())

---------- primary ----------
Thank you for the thoughtful feedback! I'm glad you enjoyed the Haiku. Here's a revision that adds a personal touch:

Cobblestones shine bright,  
Autumn's breath hums soft and low—  
Paris sighs with me.  

This version maintains the core imagery while introducing a sense of shared experience and emotion. If you'd like further adjustments, feel free to let me know!
---------- critic ----------
Your revised Haiku beautifully integrates the personal connection, making it even more engaging for readers. The phrase "Paris sighs with me" adds emotional depth and a sense of companionship with the city's weather, which is wonderfully effective. 

Feedback: The Haiku successfully evokes vivid imagery and emotion, making it relatable and evocative. There isn't much to improve in this refined version, but ensure the rhythm flows naturally as some variations might slightly alter the original 5-7-5 syllable structure traditionally found in Haikus. 

Otherwise, it's a

TaskResult(messages=[TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=219, completion_tokens=79), content="Thank you for the thoughtful feedback! I'm glad you enjoyed the Haiku. Here's a revision that adds a personal touch:\n\nCobblestones shine bright,  \nAutumn's breath hums soft and low—  \nParis sighs with me.  \n\nThis version maintains the core imagery while introducing a sense of shared experience and emotion. If you'd like further adjustments, feel free to let me know!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=318, completion_tokens=119), content='Your revised Haiku beautifully integrates the personal connection, making it even more engaging for readers. The phrase "Paris sighs with me" adds emotional depth and a sense of companionship with the city\'s weather, which is wonderfully effective. \n\nFeedback: The Haiku successfully evokes vivid imagery and emotion, making it relatable and evocative. There isn\'

The team continued from where it left off, allowing the primary agent to respond to the feedback.

Next, let's show how termination conditions can be combined using the AND (`&`) and OR (`|`) operators to create more complex termination logic. For example, we'll create a team that stops either after 10 messages are generated or when the critic agent approves a message.


In [5]:
max_msg_termination = MaxMessageTermination(max_messages=10)
text_termination = TextMentionTermination("APPROVE")
combined_termination = max_msg_termination | text_termination

round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Misty dawns unfold,  
Seine's whispers in the crisp air—  
Paris draped in gold.
---------- critic ----------
This Haiku beautifully captures the serene and enchanting atmosphere of Paris through its weather. The opening line, "Misty dawns unfold," evokes a sense of calm and beginnings, setting the stage for a picturesque morning. "Seine's whispers in the crisp air" effectively personifies the river, adding a gentle auditory element that further immerses the reader. Finally, "Paris draped in gold" presents a striking visual, suggesting the warm tones of a city bathed in sunlight.

Feedback: The Haiku is vivid and emotive, however, consider exploring a slightly different aspect or mood of Parisian weather to make it even more unique. Perhaps referencing a specific seasonal change or unexpected weather twist could add another layer of depth.

Overall, this is a captivating depiction.

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=489, completion_tokens=24), content="Misty dawns unfold,  \nSeine's whispers in the crisp air—  \nParis draped in gold.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=487, completion_tokens=158), content='This Haiku beautifully captures the serene and enchanting atmosphere of Paris through its weather. The opening line, "Misty dawns unfold," evokes a sense of calm and beginnings, setting the stage for a picturesque morning. "Seine\'s whispers in the crisp air" effectively personifies the river, adding a gentle auditory element that further immerses the reader. Finally, "Paris draped in gold" presents a striking visual, suggesting the warm tones of a city bathed in sunlight.\n\nFeedback: The Haiku is vivid and emotive, however, 

The conversation stopped after the critic agent approved the message, although it could have also stopped if 10 messages were generated.

Alternatively, if we want to stop the run only when both conditions are met, we can use the AND (`&`) operator.

In [6]:
combined_termination = max_msg_termination & text_termination